In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import laspy

import random
import math

In [ ]:
# GET GROUND HEIGHT

las = laspy.read('data/banska4.las')
ground_data = las[las.classification == 2] 
points_data = np.stack([ground_data.Z], axis = 0).transpose((1,0))

points_sum = 0
data_min = min(points_data)

for i in range(len(points_data)):
    points_sum += points_data[i][0] - data_min[0]

avg_ground_height = math.floor(points_sum / len(points_data) + data_min[0])

In [ ]:
# GET TREE EPS

tree_data = las[las.classification == 5]
points_data = np.stack([tree_data.Z], axis = 0).transpose((1,0))

points_data = sorted(points_data, key=lambda x: x[0], reverse=True)
points_data = points_data[:100]

points_sum = 0
data_min = min(points_data)

for i in range(len(points_data)):
    points_sum += points_data[i][0] - data_min[0]

avg_tree_height = math.floor(points_sum / len(points_data) + data_min[0])
tree_eps = (avg_tree_height - avg_ground_height) // 4

In [64]:
# FIRST CLUSTERING

points_data = np.stack([tree_data.X,tree_data.Y,tree_data.Z], axis = 0).transpose((1,0))
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(points_data)
geom = geom.remove_radius_outlier(3,1000)[0]

labels = np.array(geom.cluster_dbscan(tree_eps, 6))
random_colors = []
for label in labels:
    if label == -1:
        random_colors.append([0,0,0])
    else:    
        random_colors.append([random.random()*0.8 +0.2,random.random()*0.8 +0.2,random.random()*0.8 +0.2])
colors = [random_colors[x] for x in labels]
geom.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([geom])

In [ ]:
# GET SIZE OF EACH CLUSTER

cluster_sizes = []
for i in set(labels):
   cluster_sizes.append((i, labels.tolist().count(i)))
#plt.figure()
#plt.plot([x for x in range(max(labels)+2)],[x[1] for x in cluster_sizes])
#plt.show()

In [39]:
# SEPARATE WELL AND SHITTY CLUSTERED POINTS WITH LABELS

clusters_sorted = sorted(cluster_sizes,key=lambda x: x[1],reverse=True)
clusters_sizes_sorted = [x[1] for x in clusters_sorted]
avg_cluster_size = sum(clusters_sizes_sorted) / len(clusters_sizes_sorted)

well_clustered_points = []
well_clustered_labels = []

shitty_clustered_points = []
shitty_clustered_labels = []

all_points = np.asarray(geom.points)

for i in range(len(all_points)):
    for cluster in clusters_sorted:
        if labels[i] == cluster[0]:
            if cluster[1] > avg_cluster_size * 3:
                shitty_clustered_points.append(all_points[i])
                shitty_clustered_labels.append(labels[i])
            else:
                well_clustered_points.append(all_points[i])
                well_clustered_labels.append(labels[i])         


pointcloud_final = o3d.geometry.PointCloud()
pointcloud_final.points = o3d.utility.Vector3dVector(well_clustered_points)
o3d.visualization.draw_geometries([pointcloud_final])

In [79]:

big_cluster = o3d.geometry.PointCloud()

for label in set(shitty_clustered_labels):
    one_shitty_cluster_points = []
    for i in range(len(shitty_clustered_points)):
        if shitty_clustered_labels[i] == label:
            one_shitty_cluster_points.append(shitty_clustered_points[i])
    
    big_cluster.points = o3d.utility.Vector3dVector(one_shitty_cluster_points)
    labels_round_2 = np.array(big_cluster.cluster_dbscan(tree_eps/2, 8))

    cluster_sizes2 = []
    for i in set(labels_round_2):
        cluster_sizes2.append((i, labels_round_2.tolist().count(i)))

    random_colors2 = []
    for l in labels_round_2:
        if l == -1:
            random_colors2.append([0,0,0])
        else:    
            random_colors2.append([random.random()*0.8 +0.2,random.random()*0.8 +0.2,random.random()*0.8 +0.2])
    colors2 = [random_colors2[x] for x in labels_round_2]
    big_cluster.colors = o3d.utility.Vector3dVector(colors2)

    o3d.visualization.draw_geometries([big_cluster])



[(0, 49), (1, 12), (2, 8), (3, 9), (4, 8), (5, 25), (6, 22), (7, 9), (8, 48), (9, 25), (10, 13), (11, 14), (12, 7), (13, 8), (14, 16), (15, 12), (16, 6), (17, 40), (18, 40), (19, 31), (20, 10), (21, 14), (22, 12), (23, 21), (24, 183), (25, 11), (26, 8), (27, 45), (28, 138), (29, 57), (30, 133), (31, 32), (32, 133), (-1, 278)]
1477
[(0, 1509), (1, 21), (2, 74), (-1, 67)]
1671
[(0, 10), (1, 12), (2, 13), (3, 23), (4, 34), (5, 52), (6, 10), (7, 12), (8, 8), (9, 18), (10, 17), (11, 21), (12, 30), (13, 303), (14, 15), (15, 44), (16, 9), (17, 37), (18, 13), (-1, 128)]
809
